In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://raw.githubusercontent.com/suchy1713/EmotionPredicition/master/data/preprocessed/data.csv')
data = df.values
X = data[:, :11]
Y = data[:, 11:]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [0]:
#zapraszam do zabawy